<a href="https://colab.research.google.com/github/pasumarthi/Backprop/blob/master/ADR_tweet_classifier_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The dataset given has list of tweets and corresponding labels of ADR or non-ADR, with 0 or 1 respectively. To solve this problem, I consider it as a binary-classification problem with tweets as independent variable and labels of 1 and 0 as dependant variable. This can be seen as an NLP based unstructured data, with each tweet is given as input to a time series recurrent model(Ex:LSTM). The LSTM model we need is many-to-one type, with many input time steps of words in the tweet, and one output whether the given tweet is ADR or non-ADR.

Dataset given has only around 23,500 tweets and most of the tweets are repeating multiple times, which is a small dataset to learn the structure of english language and on top of that learn medical drugs related symntic understanding.
Since this is what is made available, I will try the best methods avaliable and simple libraries that can make it possible.

I have used keras library with tensorflow backend. It is very efficient, easy to implement and easy to understand. For sequence modeling, sentance with word time steps, I have  used LSTM(bidirectional). Which is the best sequence model available. I will use this jupyter notebook to present my code along with explanation of the code at every significant step, to justify  my choice of code and logic.

Following two cells, download the "glove" embedding vectors dictionary and I am using "glove.6B.100d.txt" this perticulary dictionary. It is trained on 6 Billion words and each word is represented as 100 dimentional long vector.

**If you are facing any problem in running this notebook, revert back to me with the issue you are facing.**
my mail id: chunduri11@gmail.com

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [30]:
ls

Data.csv           glove.6B.200d.txt  glove.6B.50d.txt  sample_data/
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip


https://drive.google.com/file/d/1sOwwjLXsCiFBhCfrYaDOCi81fcOinWQc/view?usp=sharing
use the above link to get the dataset "Date.csv"

In [4]:
# load dataset "Data.csv" from local computer using these two lines of code
from google.colab import files
files.upload()
!ls

Saving Data.csv to Data.csv
Data.csv  sample_data


In [5]:
# glove.6B.zip, download using this link 
!wget --header="Host: nlp.stanford.edu" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-US,en;q=0.9" --header="Cookie: _ga=GA1.2.456156586.1539718115; _gid=GA1.2.491677602.1539718115; _gat=1" --header="Connection: keep-alive" "https://nlp.stanford.edu/data/glove.6B.zip" -O "glove.6B.zip" -c


--2018-11-05 12:37:23--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  17.8MB/s    in 43s     

2018-11-05 12:38:06 (19.3 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [6]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [24]:
!ls

Data.csv	   glove.6B.200d.txt  glove.6B.50d.txt	sample_data
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip


In [0]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using pandas, load the dataset and islolate the two columns that are used for training, "**Tweet**" and "**ADR_label**". Rest of the id columns are not needer for training, which can be ignored. Using keras "**Tokenizer**", I created tokens of unique vocabulary dictionary and create tokenized version of each tweet with a sequence of integers. "X_t" and "X_te" are the train and test input data after tokenization. Here it is a list of tweets in tokenized format(with integer numbe for each word). "**pad_sequences**" function takes all the tokenized tweets, using "**maxlen**" ensured that all the tweets are of same length. By **padding** tweets that are shorted and **truncating** tweets which are longer.

In [32]:
df = pd.read_csv('Data.csv')
#train.isnull().any(),test.isnull().any()
X = df.Tweet[:5000]
print(type(X))
y= df.ADR_label[:5000]
#train test split, 20% for test set, rest for train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



list_sentences_train = X_train
list_sentences_test = X_test
type(list_sentences_train)



# tokenize the sentence by replacing words with equalent integer sequence. Each word gets a unique integer value.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

print(len(tokenizer.word_index)) #unizue words vocabulary from the dataset, this is a dictionary with word as key and a sequence integer as value.

vocab_size = len(tokenizer.word_index) + 1
maxlen = 40
# pad_sequences -- pads a sentance if it is less than "maxlen", truncates it if greater than "maxlen"
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)



totalNumWords = [len(one_comment) for one_comment in list_tokenized_train]
#totalNumWords

<class 'pandas.core.series.Series'>
5795


In this cell, I open the "glove" dictionary file and save the "words" and corresponding "vectors" in a dictionary "**embeddings_index**" of keys as words and vectors as values. This dictionary is converted to a matrix with **rows** equal to total unique words(**vocab_size**) from from all our tweets, **columns** equal to dimention(100d) of embedding vector. what I did is, essentially used the glove english language dictionary and selected corresponding word vector for each of the unique words from my database. Now all our words in our tweet database will have a prior or pretrained vectors which understand the symantics of English.

In [33]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs #dictinary of all the words and vectors as keys and values of "glove.6B.100d.txt"
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector #vector matrix of all the unique words from the dataset(all tweets)(size is (16055,100))
    

Loaded 400000 word vectors.


In [18]:
print(embedding_matrix.shape)

(5806, 300)


Now we have our Deep Learning model stacked according the sequence of layers. First layer is "**Input layer**", Sencond layer is Embedding layer, Third layer is "**biLSTM layer**", Fourth layer is "**GlobalMaxPooling1D**", followed by **Dropout**, **Dence**, **Dropout** and **Dence** layers respectively. Finally, we feed the output into a **Sigmoid** layer. The reason why sigmoid is used is because we are trying to achieve a binary classification(1,0) , and the sigmoid function will squash the output between the bounds of 0 and 1.

 **Embedding layer**, we have to pass the "embedding_matrix" as pretrained vector matrix. Make the Embedding layer trainable to fine tune the word vectors to understand the medical drugs terminology on top of English symantics.

In [28]:
inp = Input(shape=(maxlen, )) 
embed_size = 100
x = Embedding(vocab_size, embed_size, weights=[embedding_matrix], trainable=True)(inp)
#x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
#model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
x = Bidirectional(LSTM(50, return_sequences=False, dropout=0.1, recurrent_dropout=0.1))(x)
#x = GlobalMaxPool1D()(x)
#x = Dropout(0.5)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1, activation="sigmoid")(x)

ValueError: ignored

Now the model is defined. In the next cell, first line of code createas an object of the defined Model from the last cell. "model.compile" takes three important things loss function, optimizer and metric. Follwed by the next cell where training starts.

In [21]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 40)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 40, 300)           1741800   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 40, 100)           140400    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 100)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_6 (Dropout)          (None, 50)                0         
__________

In [22]:
print(len(X_t),len(y_train))
print(len(X_te),len(y_test))

4000 4000
1000 1000


In [23]:
# training the model for 10 epochs
batch_size = 32
epochs = 10
model.fit(X_t,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Train on 3600 samples, validate on 400 samples
Epoch 1/10
3600/3600 [==============================] - 26s 7ms/step - loss: 0.0462 - acc: 0.9881 - val_loss: 1.9801e-05 - val_acc: 1.0000
Epoch 2/10
3600/3600 [==============================] - 24s 7ms/step - loss: 4.1384e-04 - acc: 1.0000 - val_loss: 1.9240e-06 - val_acc: 1.0000
Epoch 3/10
3600/3600 [==============================] - 24s 7ms/step - loss: 1.8342e-04 - acc: 1.0000 - val_loss: 4.6343e-07 - val_acc: 1.0000
Epoch 4/10
3600/3600 [==============================] - 24s 7ms/step - loss: 1.2669e-04 - acc: 1.0000 - val_loss: 1.6660e-07 - val_acc: 1.0000
Epoch 5/10
3600/3600 [==============================] - 24s 7ms/step - loss: 5.8435e-05 - acc: 1.0000 - val_loss: 1.3351e-07 - val_acc: 1.0000
Epoch 6/10
3600/3600 [==============================] - 24s 7ms/step - loss: 1.2682e-04 - acc: 1.0000 - val_loss: 1.2398e-07 - val_acc: 1.0000
Epoch 7/10
3600/3600 [==============================] - 24s 7ms/step - loss: 3.6477e-05 - acc: 1.00

In [57]:
# test set evaluation
model.evaluate(X_te,y_test)

100/100 [==============================] - 0s 3ms/step


[5.7153961970470844e-05, 1.0]

Total recods available are 23.5k, 20% of these are allocated for test set, using "train_test_split" of sklearn. During training among the data made available for training 10% is set aside for validation set, which can be seen as validation loss and validation accuracy during the training. This practice of dividing data into train, validationa and test a very good practice.

the last step "model.evaluate" used the test set(20%) is to test trained model on an endependant data.

- performace accuracy and loss for pre-trained embedding vector model: 
- loss = 0.294
- accuracy = 89.15%

In [63]:
a  = [[1,2,3,4,5],[11,22,33,44,55],[111,222,333,444,555]]
a[:][1:4]

[[11, 22, 33, 44, 55], [111, 222, 333, 444, 555]]